<a href="https://colab.research.google.com/github/psaw/hse-ai24-ml/blob/main/LogReg_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [243]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [244]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [245]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [246]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?  
**Ответ:**  `Region`

In [247]:
# ваш код здесь
data.describe(include='object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [248]:
# ваш код здесь
cat2_cols = []
for col in data.columns:
    if data[col].dtype == 'object' and data[col].nunique() == 2:
        cat2_cols.append(col)

print(cat2_cols)  # ['Marital Status', 'Gender', 'Home Owner', 'Purchased Bike']

['Marital Status', 'Gender', 'Home Owner', 'Purchased Bike']


In [249]:
for col in cat2_cols:
    mod = data[col].mode()[0]  # "мода" - самое частое значение
    data[col] = data[col].apply(lambda x: 1 if x == mod else 0)

In [250]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                1000 non-null   int64 
 1   Marital Status    1000 non-null   int64 
 2   Gender            1000 non-null   int64 
 3   Income            1000 non-null   int64 
 4   Children          1000 non-null   int64 
 5   Education         1000 non-null   object
 6   Occupation        1000 non-null   object
 7   Home Owner        1000 non-null   int64 
 8   Cars              1000 non-null   int64 
 9   Commute Distance  1000 non-null   object
 10  Region            1000 non-null   object
 11  Age               1000 non-null   int64 
 12  Purchased Bike    1000 non-null   int64 
dtypes: int64(9), object(4)
memory usage: 101.7+ KB


Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?  
**Ответ:** 4

In [251]:
len(data.select_dtypes(include='object').columns)  # осталось 4 категориальных признака. предыдущие конвертировались в int64

4

In [252]:
data.drop(columns=data.select_dtypes(include='object').columns, axis=1, inplace=True)

Удалите столбец `ID`, так как он по сути нечисловой.

In [253]:
# ваш код здесь
data.drop('ID', axis=1, inplace=True)

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [254]:
# ваш код здесь
X = data.drop('Purchased Bike', axis=1)
y = data['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [255]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?  
**Ответ:** 700

In [256]:
# ваш код здесь
Xtrain.shape

(700, 7)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [257]:
from sklearn.preprocessing import MinMaxScaler

In [258]:
# ваш код здесь
mmscaler = MinMaxScaler()
mmscaler.fit(Xtrain)
Xtrain_scaled = pd.DataFrame(mmscaler.transform(Xtrain), columns=mmscaler.get_feature_names_out())
Xtest_scaled = pd.DataFrame(mmscaler.transform(Xtest), columns=mmscaler.get_feature_names_out())

Теперь обучите логистическую регрессию на тренировочных данных

In [259]:
from sklearn.linear_model import LogisticRegression

In [260]:
# ваш код здесь
lr = LogisticRegression()
lr.fit(Xtrain, ytrain)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [261]:
# ваш код здесь
pred_train = lr.predict(Xtrain_scaled)
pred_test = lr.predict(Xtest_scaled)

Оцените значение accuracy на трейне и на тесте.

In [262]:
from sklearn.metrics import accuracy_score

In [263]:
# ваш код здесь
acc_train = accuracy_score(ytrain, pred_train)
acc_test = accuracy_score(ytest, pred_test)
print(f"Accuracy (train): {acc_train:.3f}")
print(f"Accuracy (test): {acc_test:.3f}")

Accuracy (train): 0.594
Accuracy (test): 0.513


Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [264]:
from sklearn.preprocessing import PolynomialFeatures

# ваш код здесь
poly = PolynomialFeatures(degree=2)
poly.fit(Xtrain)

# Xtrain_poly = pd.DataFrame(poly.transform(Xtrain), columns=poly.get_feature_names_out())
# Xtest_poly = pd.DataFrame(poly.transform(Xtest), columns=poly.get_feature_names_out())
Xtrain = pd.DataFrame(poly.transform(Xtrain), columns=poly.get_feature_names_out())
Xtest = pd.DataFrame(poly.transform(Xtest), columns=poly.get_feature_names_out())
Xtrain_poly = Xtrain
Xtest_poly = Xtest

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?  
**Ответ:** на 29


In [265]:
len(Xtrain_poly.columns)

# заново применим масштабирование
mmscaler.fit(Xtrain_poly, ytrain)
Xtrain_poly_scaled = mmscaler.transform(Xtrain_poly)
Xtest_poly_scaled = mmscaler.transform(Xtest_poly)

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [267]:
# ваш код здесь
lr2 = LogisticRegression()
lr2.fit(Xtrain_poly_scaled, ytrain)
pred2_train = lr2.predict(Xtrain_poly_scaled)
pred2_test = lr2.predict(Xtest_poly_scaled)
acc2 = accuracy_score(ytest, pred2_test)

print(f"Accuracy (test): {acc2:.3f}")

Accuracy (test): 0.617


**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

In [268]:
round(acc2 - acc_test, 4)

0.1033

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [269]:
from sklearn.metrics import classification_report

In [270]:
# ваш код здесь
print(classification_report(ytest, pred_test))

              precision    recall  f1-score   support

           0       0.55      0.22      0.31       152
           1       0.50      0.82      0.62       148

    accuracy                           0.51       300
   macro avg       0.53      0.52      0.47       300
weighted avg       0.53      0.51      0.47       300



In [271]:
print(classification_report(ytest, pred2_test))

              precision    recall  f1-score   support

           0       0.65      0.54      0.59       152
           1       0.60      0.70      0.64       148

    accuracy                           0.62       300
   macro avg       0.62      0.62      0.61       300
weighted avg       0.62      0.62      0.61       300



Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [272]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 36), (210, 36))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [285]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

lr3 = LogisticRegression()
lr3.fit(XtrainS, ytrainS)
pred3_prob = lr3.predict_proba(Xval)

for thr in np.arange(0, 1, 0.01):
    # ваш код здесь
    pred3_thr = (pred3_prob[:, 1] >= thr)*1
    acc = accuracy_score(yval, pred3_thr)
    rec = recall_score(yval, pred3_thr)

    if rec >= RecMax and acc >= 0.6:
        RecMax = rec
        BestThr = thr
        BestAcc = acc

# print(recall_score(yval, pred3))
print (BestThr, RecMax, BestAcc)

0.44 0.8240740740740741 0.6


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [290]:
# ваш код здесь
lr4 = LogisticRegression()
lr4.fit(Xtrain, ytrain)
pred4_prob = lr4.predict_proba(Xtest)

pred4_thr = (pred4_prob[:, 1] >= BestThr)*1


print(f"Recall = {recall_score(ytest, pred4_thr):.3f}, Accuracy = {accuracy_score(ytest, pred4_thr):.3f}")

Recall = 0.818, Accuracy = 0.563


**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.